In [ ]:
#sample data from: https://www.kaggle.com/gunnvant/game-of-thrones-srt

import json
import pandas as pd

#find a different file from kaggale (json) file that I want to use
with open("season1.json", "r") as read_file:
    data = json.load(read_file)

data_frame = pd.DataFrame(data)
data_frame

In [ ]:
#reorganize (swap the axis) so the data makes more sense and is easier to read
reordered = data_frame.T
reordered

In [ ]:
#get bloean value for things that are lines versus empty boxes (get length in terms of diologue in each epsiode)
reordered.notna()

#how many lines of talking per each episode
reordered.count(axis = 1)

In [ ]:
#make a visualization of the above
reordered.count(axis=1).plot()

In [ ]:
#make is a more readable graph by using a bar graph with specific labels - more meanigful visaulization
reordered.count(axis=1).plot(kind='bar')

Next assignment - demo

In [ ]:
#going from source code to pandas (can use beautiful soup to scrape just about anything)
#documented at: https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.metacritic.com/game/switch/animal-crossing-new-horizons/user-reviews?page=0'

#be a "user" so the site lets you in to scrape (sneaky)
user_agent = {'User-agent': 'Mozilla/5.0'}
response = requests.get(url, headers = user_agent)

soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
#box to put structured data in (dictionary format to store values to hold info for each review)
review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}

In [ ]:
#look for review_content tags since every user review has this tag
 
#break at empty boxes 
for review in soup.find_all('div', class_='review_content'): 
    if review.find('div', class_='name') == None:
        break 
    #find and save review - "append" because its adding to the data frame
    review_dict['name'].append(review.find('div', class_='name').find('a').text)
    review_dict['date'].append(review.find('div', class_='date').text)
    review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
    #blurb vs blurb expanded & do not display "no review data"
    if review.find('span', class_='blurb blurb_expanded'): 
        review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text)
        print(review.find('span', class_='blurb blurb_expanded').text)
    elif review.find('div',class_='review_body').find('span') == None:
        review_dict['review'].append('No review text.')
        print("No review")
    else:
        review_dict['review'].append(review.find('div',class_='review_body').find('span').text)
        print(review.find('div',class_='review_body').find('span').text)

In [ ]:
#can we take this to a data frame? send data to panda frame
ac_reviews = pd.DataFrame(review_dict)

In [ ]:
#show it in the final form?
ac_reviews

Next class demo

In [ ]:
#network graph (connection of things) - math! 
#edges are the connections, nodes are the things

V = {1, 2, 3, 4, 5}
E = {(1, 2), (1, 4), (2, 5), (3, 4), (4, 5)}

import networkx as nx

G = nx.Graph()
G.add_nodes_from(V)
G.add_edges_from(E)

import matplotlib.pyplot as plt

nx.draw_networkx(G, font_color="white")
plt.axis('off');

In [ ]:
#

G_symmetric = nx.Graph()
G_symmetric.add_edge('Elizabeth Bennet','Fitzwilliam Darcy')
G_symmetric.add_edge('Elizabeth Bennet','Jane Bennet')
G_symmetric.add_edge('Elizabeth Bennet','Georgiana Darcy')
G_symmetric.add_edge('Elizabeth Bennet','Charles Bingley')
G_symmetric.add_edge('Fitzwilliam Darcy','Jane Bennet')
G_symmetric.add_edge('Fitzwilliam Darcy','Georgiana Darcy')
G_symmetric.add_edge('Fitzwilliam Darcy','Charles Bingley')
G_symmetric.add_edge('Charles Bingley','Jane Bennet')

nx.draw_networkx(G_symmetric)

introducing tei - how do we make sense of complex meta data? 

In [ ]:
#parcer for xml - make sense of it
import lxml.etree

tree = lxml.etree.parse('Shrew.xml')

print(tree)

In [ ]:
#navigation 
print(lxml.etree.tostring(tree).decode()[0:500])

In [ ]:
#to read tree we have to understand roots and children
#repeats name space "announces itself to you"
print(tree.getroot().tag)

print(len(tree.getroot()))

#looping - for every piece here "what are you?"
for child in tree.getroot():
    print(child.tag, child.attrib)
    for nested in child:
        print(nested.tag, nested.attrib)

In [ ]:
#start seeing divs that coorespond to those nodes 
start = tree.getroot()[1][1]
print(start.tag)
for child in start:
    print (child.tag)

In [ ]:
#this won't work - need the namespace (as shown above)
#save name space map first and then use it to navigate the tree

print(tree.getroot().find('title'))

#using a namespace map

NSMAP = {'tei': 'http://www.tei-c.org/ns/1.0'}
print(tree.getroot().find('.//tei:title', namespaces=NSMAP).text)

In [ ]:

def character_network(tree):
    """Construct a character interaction network.

    Construct a character interaction network for Shakespeare texts in
    the Folger Digital Text collection. Character interaction networks
    are constructed on the basis of successive speaker turns in the texts,
    and edges between speakers are created when their utterances follow
    one another.

    Arguments:
        tree: An lxml.ElementTree instance representing one of the XML
            files in the Folger Shakespeare collection.

    Returns:
        A character interaction network represented as a weighted,
        undirected NetworkX Graph.

    """
    #creating a graph
    G = nx.Graph()
    # extract a list of speaker turns for each scene in a play
    for scene in tree.iterfind('.//tei:div2[@type="scene"]', NSMAP):
        speakers = scene.findall('.//tei:sp', NSMAP)
        # iterate over the sequence of speaker turns...
        for i in range(len(speakers) - 1):
            # ... and extract pairs of adjacent speakers
            try:
                speaker_i = speakers[i].attrib['who'].split('_')[0].replace('#', '')
                speaker_j = speakers[i + 1].attrib['who'].split('_')[0].replace('#', '')
                # if the interaction between two speakers has already
                # been attested, update their interaction count
                if G.has_edge(speaker_i, speaker_j):
                    G[speaker_i][speaker_j]['weight'] += 1
                # else add an edge between speaker i and j to the graph
                else:
                    G.add_edge(speaker_i, speaker_j, weight=1)
            except KeyError:
                continue
    return G

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = character_network(tree.getroot())
print(f"N nodes = {G.number_of_nodes()}, N edges = {G.number_of_edges()}")

In [ ]:
import collections

interactions = collections.Counter()

for speaker_i, speaker_j, data in G.edges(data=True):
    interaction_count = data['weight']
    interactions[speaker_i] += interaction_count
    interactions[speaker_j] += interaction_count

nodesizes = [interactions[speaker] * 5 for speaker in G]

In [ ]:
# Create an empty figure of size 15x15
fig = plt.figure(figsize=(15, 15))
# Compute the positions of the nodes using the spring layout algorithm
pos = nx.spring_layout(G, k=0.5, iterations=200)
# Then, add the edges to the visualization
nx.draw_networkx_edges(G, pos, alpha=0.4)
# Subsequently, add the weighted nodes to the visualization
nx.draw_networkx_nodes(G, pos, node_size=nodesizes, alpha=0.4)
# Finally, add the labels (i.e. the speaker IDs) to the visualization
nx.draw_networkx_labels(G, pos)
plt.axis('off');

In [ ]:
#save the above as a JSON file
import json
from networkx.readwrite import json_graph

with open('shrew.json', 'w') as f:
    json.dump(json_graph.node_link_data(G), f)

with open('shrew.json') as f:
    d = json.load(f)

G = json_graph.node_link_graph(d)
print(f"Graph with {len(G.nodes())} nodes and {len(G.edges())} edges.")